In [ ]:
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import client as cp
import middleware as mp

# Matplot lib settings
%matplotlib notebook
matplotlib.rcParams.update({
        'font.size': 11,
        'font.family': 'sans-serif',
        'xtick.labelsize': 10,
        'ytick.labelsize': 10
    })

# Numpy settings
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [ ]:
base_dir = '/Users/ruifengxu/Development/asl-fall16-project/logs/trace/'
client_log_template = 'client-{}.log'
middleware_log_template = 'trace.log'

In [ ]:
def process_client_data(thread_pool_size, num_clients, num_client_machines, start, duration):
    end = start + duration
                
    client_tp = 0
            
    client_rts = np.zeros(num_client_machines * duration)
    client_rts_weight = np.zeros(num_client_machines * duration)
    client_rts_index = 0
    client_rts_bucket = None

    for log_id in range(1, num_client_machines + 1, 1):
        filename = os.path.join(
            base_dir,
            client_log_template.format(log_id)
        )
        data = cp.parse_log(filename)

        num_ops = data['all_global'][end - 2]['num_ops'] - data['all_global'][start - 2]['num_ops']
        
        client_tp += (num_ops // duration)
                    
        for entry in data['all_local'][start - 1: end - 1]:
            client_rts[client_rts_index] = entry['rt_mean']
            client_rts_weight[client_rts_index] = entry['num_ops']
            client_rts_index = client_rts_index + 1

        if client_rts_bucket is None:
            client_rts_bucket = list(data['all_bucket'])
        else:
            client_rts_bucket = [x + y for x, y in zip(client_rts_bucket, data['all_bucket'])]
        
    client_rt_mean = np.average(client_rts, weights=client_rts_weight)
    client_rt_low = cp.get_percentile(client_rts_bucket, 5)
    client_rt_high = cp.get_percentile(client_rts_bucket, 95)
            
    print('{:6.0f} {:6.3f} {:6.3f} {:6.3f} {:6.3f}'.format(
            client_tp,
            client_rt_mean, client_rt_low, client_rt_high,
            1 / client_rt_mean * 1000 * num_clients * num_client_machines / client_tp))    
    
    return {
        'tp_mean': client_tp,
        'tp_std': 0,
        'tp_ci': 0,
        'rt_mean': client_rt_mean,
        'rt_low': client_rt_low,
        'rt_high': client_rt_high
    }

In [ ]:
client = process_client_data(16, 64, 3, 601, 4200)
client

In [ ]:
def process_middleware_data(start, duration):
    read_request_times = []
    queue_times = []
    send_request_times = []
    read_response_times = []
    send_response_times = []
    total_times = []
        
    filename = os.path.join(base_dir, middleware_log_template)
    data = mp.parse_log(filename, request_type='g')
    start_time = data[0][2] + start
    end_time = start_time + duration
    
    for entry in data:                                                
        if entry[2] < start_time:
            continue
        if entry[2] == end_time:
            break
        if entry[6] is False:
            continue
        read_request_times.append(entry[8])
        queue_times.append(entry[9])
        send_request_times.append(entry[10])
        read_response_times.append(entry[11])
        send_response_times.append(entry[12])
        total_times.append(np.sum(entry[8:13])) 
    
    return {
        'read_request': np.mean(read_request_times) / 1000000, 
        'queue': np.mean(queue_times) / 1000000, 
        'forward_request': np.mean(send_request_times) / 1000000, 
        'server': np.mean(read_response_times) / 1000000, 
        'send_response': np.mean(send_response_times) / 1000000, 
        'total': np.mean(total_times) / 1000000
    }

In [ ]:
middleware = process_middleware_data(601, 4200)
print(middleware['read_request'])
print(middleware['queue'])
print(middleware['forward_request'])
print(middleware['server'])
print(middleware['send_response'])
print(middleware['total'])

In [ ]:
def get_middleware_operation_percentiles(percentile_config, start, duration):
    read_request_times = []
    queue_times = []
    send_request_times = []
    read_response_times = []
    send_response_times = []
    total_times = []

    filename = os.path.join(base_dir, middleware_log_template)

    data = mp.parse_log(filename, request_type='g')
    start_time = data[0][2] + start
    end_time = start_time + duration
    for entry in data:                                                
        if entry[2] < start_time:
            continue
        if entry[2] == end_time:
            break
        if entry[6] is False:
            continue

        read_request_times.append(entry[8])
        queue_times.append(entry[9])
        send_request_times.append(entry[10])
        read_response_times.append(entry[11])
        send_response_times.append(entry[12])
        total_times.append(np.sum(entry[8: 13])) 

    return {
        'read_request': np.percentile(read_request_times, percentile_config) / 1000000,
        'queue': np.percentile(queue_times, percentile_config) / 1000000,
        'forward_request': np.percentile(send_request_times, percentile_config) / 1000000,
        'server': np.percentile(read_response_times, percentile_config) / 1000000,
        'send_response': np.percentile(send_response_times, percentile_config) / 1000000,
        'total': np.percentile(total_times, percentile_config) / 1000000
    }

In [ ]:
percentiles = get_middleware_operation_percentiles([50, 99], 601, 4200)
print(percentiles['read_request'])
print(percentiles['queue'])
print(percentiles['forward_request'])
print(percentiles['server'])
print(percentiles['send_response'])
print(percentiles['total'])

In [ ]:
l = client['tp_mean']

processing_time = (middleware['forward_request'] + middleware['server'] + middleware['send_response']) / 1000

m = 1 / processing_time * 17 * 3

r = l / m

print('lambda', l)
print('mu', m)
print('rho', l / m)
print()

print('Measured')
print('E[r]', middleware['total'])
print('E[w]', middleware['queue'])
print('E[n]', l * middleware['total'] / 1000)
print('E[nq]', l * middleware['queue'] / 1000)
print()

print('Prediction')
print('E[r]', 1 / m / (1-r) * 1000)
print('E[w]', r / m / (1-r) * 1000)
print('E[n]', r / (1 - r))
print('E[nq]', r * r / (1 - r))
print()

print('Adjusted Prediction')
print('E[r]', 1 / m / (1-r) * 1000 * 51)
print('E[w]', r / m / (1-r) * 1000 * 51)
print('E[n]', 1 / m / (1-r) * 51 * l)
print('E[nq]', r / m / (1-r) * 51 * l)
print()